# Example Postgres Queries to Pandas using SQLAlchemy 

In [1]:
# Load packages 
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import Table, MetaData, select, and_, or_

### Define Connect function

This is a general function which connects to  postgres database. 

*** IMPORTANT - run the connection.close() when finished. 

In [2]:
# default=localhost - default port=5432
def connect(user, password, db, host='localhost', port=5432):
    '''Returns a connection and a metadata object'''
    
    # We connect with the help of the PostgreSQL URL
    url = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)

    # The return value of create_engine() is our connection object
    engine = sqlalchemy.create_engine(url, client_encoding='utf8')
    
    ## Connections 
    connection = engine.connect()
    
    # We then bind the connection to MetaData()
    meta = sqlalchemy.MetaData(bind=engine, reflect=True)

    return engine, connection, meta

# Create Table in Dataset

In [3]:
c_db = pd.read_csv('./connect.csv')

In [4]:
user = c_db['user'].as_matrix()[0]
password = c_db['password'].as_matrix()[0]
db = c_db['database'].as_matrix()[0]
# defualts are fine with me. 

In [5]:
#### connect 
engine, connection, meta = connect(user, password, db)

/home/lightner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  app.launch_new_instance()


### Upload data from csv

In [6]:
# access full dataset data 
data_csv = pd.read_csv('./portal-example/Portal/data/ODI-Portal_April2018.csv', sep='|')
data_csv.head()

,Unnamed: 0,Var0,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,...,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217
0,0,NaN,2.416430,NaN,9.309007,169465.956355,NaN,3302.0,NaN,1903.712617,...,NaN,1.903713,63.718735,NaN,NaN,NaN,4.253241,NaN,99.156992,NaN
1,1,12.822017,2.639936,-0.015189,8.905180,209876.166419,-0.415615,4276.0,0.332602,1904.122829,...,0.035213,1.904123,37.236246,NaN,0.045584,-6.072028,4.181193,7.153315,99.202192,-0.072876
2,2,-55.897289,2.893730,17.296321,6.968417,158174.459103,-0.354371,4071.0,-1.691349,1452.122274,...,0.027593,1.452122,24.040807,NaN,0.043479,-25.872388,3.337864,28.440054,99.245324,-0.289607
3,3,-16.283720,3.147535,15.429666,5.479013,114075.207259,-0.402046,8937.0,-1.870101,1162.653135,...,0.025303,1.162653,14.375296,NaN,0.038593,-25.134100,1.632790,27.225987,99.283626,-0.277122
4,4,135.884474,3.378356,-21.248107,7.020349,240458.970881,-0.104042,5499.0,2.991363,1468.297798,...,-0.003459,1.468298,12.879657,NaN,-0.002770,32.308926,2.562216,-34.310984,99.280876,0.349102


In [7]:
# push dataset to postgres database
data_csv.to_sql('odi-portal-april2018', con=engine, schema='public', if_exists='replace')

# Access and Query Data

In [8]:
tablename= 'odi-portal-april2018'
schemaname = 'public'

# get full dataset
data = pd.read_sql_table(tablename, con=engine, schema =schemaname)


In [9]:
data.head()

,index,Unnamed: 0,Var0,Var1,Var2,Var3,Var4,Var5,Var6,Var7,...,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217
0,0,0,NaN,2.416430,NaN,9.309007,169465.956355,NaN,3302.0,NaN,...,NaN,1.903713,63.718735,NaN,NaN,NaN,4.253241,NaN,99.156992,NaN
1,1,1,12.822017,2.639936,-0.015189,8.905180,209876.166419,-0.415615,4276.0,0.332602,...,0.035213,1.904123,37.236246,NaN,0.045584,-6.072028,4.181193,7.153315,99.202192,-0.072876
2,2,2,-55.897289,2.893730,17.296321,6.968417,158174.459103,-0.354371,4071.0,-1.691349,...,0.027593,1.452122,24.040807,NaN,0.043479,-25.872388,3.337864,28.440054,99.245324,-0.289607
3,3,3,-16.283720,3.147535,15.429666,5.479013,114075.207259,-0.402046,8937.0,-1.870101,...,0.025303,1.162653,14.375296,NaN,0.038593,-25.134100,1.632790,27.225987,99.283626,-0.277122
4,4,4,135.884474,3.378356,-21.248107,7.020349,240458.970881,-0.104042,5499.0,2.991363,...,-0.003459,1.468298,12.879657,NaN,-0.002770,32.308926,2.562216,-34.310984,99.280876,0.349102


In [10]:
# Number of observations
len(data)

4452

### Access specific columns 

In [11]:
# First generate a 'table' 
# Create Data Table to load
tablename= 'odi-portal-april2018'

# Set table 
data_table = Table(tablename, MetaData(), autoload=True, autoload_with=engine)

# generate a select statement 
stmt_main = select([data_table])

It is essentially an empty table. Or a way to access the actual table. 

In [12]:
print(data_table)

odi-portal-april2018


Select certain columns 


In [13]:
def get_columns(stmt, vars_interest): 
    # generate a dictioanry with the variables as keys 
    dictionary= {}

    for i in vars_interest:
        dictionary[i] = []

    # place data in the dictionary
    for result in connection.execute(stmt):
        for i in vars_interest:
            dictionary[i].append(result[i])

    # turn dictionary into dataframe 
    data = pd.DataFrame(dictionary)
    return data

# generate a list of variables of interest 
vars_interest = ['Var0', 'Var1', 'Var3', 'countryname']
data = get_columns(stmt_main, vars_interest)

In [14]:
data.head()

,Var0,Var1,Var3,countryname
0,NaN,2.416430,9.309007,Afghanistan
1,12.822017,2.639936,8.905180,Afghanistan
2,-55.897289,2.893730,6.968417,Afghanistan
3,-16.283720,3.147535,5.479013,Afghanistan
4,135.884474,3.378356,7.020349,Afghanistan


### Query on the values 

To select observations conditional on the values of the columns, change the stmt function. 

In [15]:
# define new query 
stmt_query0 = stmt_main.where(
    data_table.columns.Var0.between(5,10))

# get data with new query 
data = get_columns(stmt_query0, vars_interest)

# notice that all those outside the 
data.head()

,Var0,Var1,Var3,countryname
0,8.399127,4.527917,5.853286,Afghanistan
1,7.848888,6.016861,9.862316,Afghanistan
2,6.801168,6.535829,11.393576,Afghanistan
3,6.616606,7.111214,12.910839,Afghanistan
4,7.778218,7.817901,14.340422,Afghanistan


You can also select those in a defined list, as well as make more than one query. There is also and _or option. 

In [16]:
# list of countries of interest
country_vals = ['Malawi', 'China', 'Algeria']

# multiple conditional query
stmt_query1 = stmt_main.where(and_(
    data_table.columns.countryname.in_(country_vals),
    data_table.columns.Var0.between(5,10)))

# get data with new query 
data = get_columns(stmt_query1, vars_interest)

data.head()

,Var0,Var1,Var3,countryname
0,5.018411,16.185540,13.460233,Algeria
1,5.612184,25.871599,23.678758,Algeria
2,7.471727,28.199936,21.531688,Algeria
3,5.521733,533.257098,10.393568,China
4,8.776286,1.835082,2.288001,Malawi


You can also use python operators. 

In [17]:
# list of countries of interest
country_vals = ['Malawi', 'China', 'Algeria']

# multiple conditional query
stmt_query1 = stmt_main.where(and_(
    data_table.columns.countryname == 'Malawi',
    data_table.columns.Var0 > 5.5))

# get data with new query 
data = get_columns(stmt_query1, vars_interest)

data.head()

,Var0,Var1,Var3,countryname
0,16.483664,1.776347,2.252869,Malawi
1,8.776286,1.835082,2.288001,Malawi
2,10.079264,2.033727,2.459857,Malawi
3,9.649865,2.187209,2.652827,Malawi


## Close 

In [18]:
connection.close()